In [1]:
import torch #torch 패키지에는 다차원 데이터구조 즉 Tensor가 포함되어있고, 텐서에 대해 연산을 하는 다양한 함수가 포함되어있음.
import torchvision #torchvision 패키지에는 데이터 셋, 이미지 리사이즈, 이미지 텐서 변환, 이미지 정규화등을 할 수 있고, VGGnet과 같은 이미지 분류 모델도 포함하고있음 
#torchvision.dataset.데이터명(),torchvision.model.VGG등 
import torchvision.transforms as transforms #이미지 크기를 맞춰주거나 텐서로 변환 정규화등을 실행
import torch.nn as nn #deep learning model에 필요한 모듈이 모아져있는 패키지, 신경망을 구축하기 위한 데이터 구조나 레이어 정의
#ex) ReLU와 같은 활성화함수, nn.CrossEntropyLoss()와 같은 손실함수가 포함되어있음
import torch.nn.functional as F #nn과 같은 모듈이 모아져 있음, functional은 함수, nn은 클래스임
#torch.nn은 인스턴트화를 시켜야함->저장하여 활용이 가능하다
#torch.nn.functional은 인스턴트화 시켜줄 필요없이 버로 입력값을 받아 수행가능함
import torch.optim as optim #학습에 관련된 optimizing method가 있는 패키지
#torch.optim 다양한 최적화 알고리즘을 구현하는 패키지이다. 확률적 경사하강법을 중심으로한 파라미터 최적화 알고리즘 구현
import matplotlib.pyplot as plt #시각화 패키지
import numpy as np #메트릭스, 백터, 배열등 계산.

In [2]:
# 고정
torch.manual_seed(42)
####seed를 고정해주게되면 학습할때마다 훈련데이터가 계속 바뀌면서 모델 성능이 계속변화하는 등, 혼란이 있을 수 있음

In [3]:
#########preparing data
'''
traindata=50000, class=100
testdata=10000
data shape=32x32x3
mean=array([0.50707516, 0.48654887, 0.44091784])
array([0.26733429, 0.25643846, 0.27615047])
mean=0.478
std=0.26
'''

'\ntraindata=50000, class=100\ntestdata=10000\ndata shape=32x32x3\nmean=array([0.50707516, 0.48654887, 0.44091784])\narray([0.26733429, 0.25643846, 0.27615047])\nmean=0.478\nstd=0.26\n'

In [4]:
# 고정
torch.manual_seed(42)
####seed를 고정해주게되면 학습할때마다 훈련데이터가 계속 바뀌면서 모델 성능이 계속변화하는 등, 혼란이 있을 수 있음

In [5]:
transform=transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5, 0.5, 0.5))]
)

In [6]:
train_data=torchvision.datasets.CIFAR100(root='./data', train=True, download=True,transform=transform)

Files already downloaded and verified


In [7]:
trainloader=trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2) #데이터가져오기

In [8]:
print(trainloader.dataset)

Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [9]:
test_data = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified


In [10]:
testloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False, num_workers=2) 
#test data는 shuffle X

In [11]:
#이미지를 보여주기 위한 함수
def imshow(img) :
  img = img*0.5+0.5 #unnormalize
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()

In [12]:
dataiter = iter(trainloader) #iter(호출가능한객체, 반복을 끝낼값), iter()을 적용햐쥬는 이유 optimizer를 구성하려면 최적화할 매개변수를 포함하는 iterable을 제공해야해서 iter()을 적용하여 iterator데이터 타입으로 생성
images, labels = dataiter.next()
#iter() 함수로 trainloader에 들어있는 이미지와 라벨을 꺼낼 수 있는 객체를 생성함
#next() 함수로 이미지와 데이터 라벨을 꺼내옴

In [13]:
4*4*128

2048

In [14]:
class Net(nn.Module): #신경망을 정의할때 nn.Module을 활용하여 만드는 것이 일반적
    def __init__(self): # __init__에서 함수를 정의함, super()를 통해 nn.Module의 다양한 함수들을 
        super(Net, self).__init__() #super()를 통해 클래스 상속, super(모델명,self).__init__을 통해 nn.Module을 실행시키는 코드
        self.conv1 = nn.Conv2d(3, 32, 3,padding=1) #컨볼루션 레이어->학습가능한 필터로 이루어진 부분, 필터란:이미지의 특징을 찿아내기 위한 공용파라미터
        self.batch1=nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2) #2x2, stride=2인 Maxpooling 
        self.conv2 = nn.Conv2d(32, 32, 3,padding=1)#conv2d(input,output,kernel_size)
        self.batch2=nn.BatchNorm2d(32)
        self.conv3=nn.Conv2d(32,64,3,padding=1)
        self.batch3=nn.BatchNorm2d(64)
        self.conv4=nn.Conv2d(64,64,3,padding=1)
        self.batch4=nn.BatchNorm2d(64)
        self.conv5=nn.Conv2d(64,64,3,padding=1)
        self.batch5=nn.BatchNorm2d(64)
        self.conv6=nn.Conv2d(64,128,3,padding=1)
        self.batch6=nn.BatchNorm2d(128)
        self.conv7=nn.Conv2d(128,128,3,padding=1)
        self.batch7=nn.BatchNorm2d(128)
        self.conv8=nn.Conv2d(128,128,3,padding=1)
        self.batch8=nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(128 * 4 * 4,1024) #nn.Linear(input feature, output feature),선형변환을 적용하는 모듈,최종 이미지 output이 5x5x16의 3차원인데 이를 1차원으로 펼쳐주면 총400개의 입력값이 나오고 120개의 y값으로 반환한다
        self.batch_fc1=nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512) #84개의 뉴런을 output으로 선정한 이유는 Lenet-5는 7x12 크기의 bitmap 사진을 분류하였기 때문이다.
        self.batch_fc2=nn.BatchNorm1d(512)
        self.fc3=nn.Linear(512,256)
        self.batch_fc3=nn.BatchNorm1d(256)
        self.fc4 = nn.Linear(256, 100) #최종적으로 10개의 라벨을 추출하는 세번째 fully-connected layer
        
        
    def forward(self, x): #forward에서 모델이 어떻게 학습할지 코딩
        x = F.relu(self.batch1(self.conv1(x))) #32x32x32
        x = self.pool(F.relu(self.batch2(self.conv2(x)))) #16x16x32 pooling 적용
        x = F.relu(self.batch3(self.conv3(x))) #16x16x64
        x = F.relu(self.batch4(self.conv4(x))) #16x16x64
        x = self.pool(F.relu(self.batch5(self.conv5(x)))) #8x8x64 pooling적용
        x = F.relu(self.batch6(self.conv6(x))) #8x8x128
        x = F.relu(self.batch7(self.conv7(x)))
        x = self.pool(F.relu(self.batch8(self.conv8(x)))) #4x4x128 pooling 적용
        x = x.view(-1, 128*4*4) #conv layer를 거쳐 나온 최종 데이터=shape torch.Size([4, 16, 5, 5]), 2,3,4번째의 값을 곱한값으로 변환시킴. 첫번째 값은 배치 데이터를 의미하기 때문에 곱해주지않음
        # x = torch.flatten(x, 1) #3차원->1차원으로 평탄화작업, flatten을 해주는 이유 분류를 위한 학습 레이어에서는 1차원 데이터로 바꾸어서 학습이 되어야하기 때문이다.
        #x=torch.flatten(x,1)
        #x.size() #torch.Size([4, 400])
        x = F.relu(self.fc1(x)) #fully-connected층(1차원변환데이터를 통해 각 범주에 속할확률구하는 층)+Relu(activation function=분류를위한 데이터 특징강조,비선형) 
        #배치당 4개의 이미지에 대한 [4,120] Tensor값 추출
        x = F.relu(self.fc2(x)) #fully-connected층(1차원변환데이터를 통해 각 범주에속학확률구하는 충)+Relu(activation function=분류를 위한 데이터 특장강조,비선형)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
        
net = Net() #여기에 

###Relu함수: 활성화 함수중 하나. +/-가 반복되는 신호에서 -흐름을 차단함.
###말 그대로 양수면 자기 자신을 반환하고, 음수면 0을 반환한다->출력값의 범위가 넓고 양수인 경우 자기 자신을 그대로 반환하기 때문에, 기울기 소실문제가 발생하지 않음
###기존 활성화 함수보다 속도가 매우빠르다
##CNN을 두가지로 나누어 본다면
##-Convolution/pooling/activation function layer: 이미지를 분할하고 분석하여 특징을 추출하는 layer
##pooling: 정보를 유지하며 픽셀의 크기를 줄이는 것
##activation function: 모델의 복잡도를 올리기 위해 비선형성 추가
##-FC(Fully-connected)/activation function: 이전 레이어 출력을 평탄화(벡터화) 이미지를 분류/예측하는데 가장 적합하게 예측->이미지를 라벨로 분류하는 것

In [15]:
criterion = nn.CrossEntropyLoss() #손실함수->예측값과 실제값 비교
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9,0.999)) #optim.Adam(model.parameters(),lr=,) variable 타입의 파라미터들을 iterable 오브젝트로 넣어줘야한다.
#optimization gradient(기울기)구해서 가중치 값을 변환시켜
#최적의 weight값을 찿는다.
#SGD의 원리는 loss함수의 미분을 이용하여 loss를 줄이는것 gradient가 -가 되도록 값을 이동시키면 언젠가 최소값을 찿을 수 있다는 아이디어
#SGD적용 속도가 매우빠름.
#learning rate=미분값을 얼만큼 이동시킬지->조금씩해야함
#momentum=이전 가중치의 업데이트값의 일정 비율을 반영하는 매개변수, batch
#betas 각각의 모멘텀 값.

In [ ]:
for epoch in range(8): # 여러번 데이터셋 반복하기

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0): #enumerate 인덱스와 원소 tuple로 반환
        # 입력값을 얻습니다. 데이타는 [입력값, 레이블]의 목록입니다.
        inputs, labels = data
        
        # 매개변수 변화도를 0으로 만듭니다.
        optimizer.zero_grad() #한번 학습이 완료되면, gradient를 0으로 만들어주어야함, 초기화하지 않을시 의도한 방향과 다른방향으로 학습할 가능성
        
        # 순전파 + 역전파 + 최적화
        outputs = net(inputs) #CNN모델 적용부분
        loss = criterion(outputs, labels) #loss계산
        loss.backward() #backpropogation에서 gradient를 계산, loss값들을 가중치들에 대해서 미분 ex)x가 변화했을때 함수g가 얼마나 변하는지 알 수 있음
        optimizer.step() #업데이트할 parameter와 learning rate 들을 step() method를 통해 업데이트 한다
        
        
        # 통계 출력
        running_loss += loss.item()
        if i % 2000 == 1999: # 매 2000번 미니배치마다 출력하기
            print('[%d, %5d] loss: %.3f' %
                  (epoch+1, i+1, running_loss / 100))
            running_loss =0.0
            
print('Finished Training')

PATH = './cifar_net.pth' #저장경로
torch.save(net.state_dict(), PATH) #저장하기

[1,  2000] loss: 91.713
[1,  4000] loss: 89.808
[1,  6000] loss: 87.902
[1,  8000] loss: 85.668
[1, 10000] loss: 83.853
[1, 12000] loss: 82.700
[2,  2000] loss: 80.725
[2,  4000] loss: 79.589
[2,  6000] loss: 78.653
[2,  8000] loss: 77.469
[2, 10000] loss: 76.833
[2, 12000] loss: 75.583
[3,  2000] loss: 73.818


In [ ]:
#print(Loss.count)

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()
#iter() 함수로 trainloader에 들어있는 이미지와 라벨을 꺼낼 수 있는 객체를 생성함
#next() 함수로 이미지와 데이터 라벨을 꺼내옴
# 이미지 출력하기

imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(10)))

net = Net() #net초기화
net.load_state_dict(torch.load(PATH)) #train으로 학습한 모델의 파라미터 불러오기 (dictionary형태로 parameter정보가 포함되어있음)
outputs = net(images) #모델적용
_, predicted = torch.max(outputs, 1) #예측최대값과 최대값에 인데스를 튜플로 반환,행을 기준으로 최대값 찿음
print('Predicted : ', ' '.join('%5s' % classes[predicted[j]] for j in range(10)))

In [ ]:
correct = 0
total = 0
with torch.no_grad(): #test예측할때는 gradient,context를 비활성화 시켜줌으로써 계산속도 향상시켜줌
    for data in testloader:
        images, labels = data
        outputs = net(images) #모델적용
        _, predicted = torch.max(outputs.data, 1) #행을 기준으로 최대값 찿음
        total += labels.size(0) #전체 10000개
        correct += (predicted == labels).sum().item() #예측값==실제값 카운트
        
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

In [ ]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images) #모델 적용
        _, predictions = torch.max(outputs, 1) #행기준으로 예측 최대값 찿음
        for label, prediction in zip(labels, predictions): #zip함수를 통해 (실제값,예측값)형태로 묶음
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.2f} %')